### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [479]:
#%%pycodestyle
from scipy.stats import mode


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif type(max_features) is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini_ind(self, r):
        return 1 - np.sum(r ** 2, axis=1)

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return self.__gini_ind(((l_c + r_c).T / (l_s + r_s)).T) - \
               ((l_c.shape[1] / l_s) * self.__gini_ind((l_c.T / l_s).T) + \
                (r_c.shape[1] / r_s) * self.__gini_ind((r_c.T / r_s).T))

    def __entrop_ind(self, r):
        return 1 - np.sum(r * np.log(r), axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return self.__entrop_ind(((l_c + r_c).T / (l_s + r_s)).T) - \
               ((l_c.shape[1] / l_s) * self.__entrop_ind((l_c.T / l_s).T) + \
                (r_c.shape[1] / r_s) * self.__entrop_ind((r_c.T / r_s).T))

    def __misclass_ind(self, r):
        return 1 - np.max(r, axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return self.__misclass_ind(((l_c + r_c).T / (l_s + r_s)).T) - \
               ((l_c.shape[1] / l_s) * self.__misclass_ind((l_c.T / l_s).T) + \
                (r_c.shape[1] / r_s) * self.__misclass_ind((r_c.T / r_s).T))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.random.permutation(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        gains_all = []
        features = []
        thresholds = []

        for i in self.get_feature_ids(x.shape[1]):
            x_sorted, y_sorted = self.__sort_samples(x[:, i], y)

            diff = np.where(x_sorted[:-1] != x_sorted[1:])[0]

            if diff.shape[0] == 0:
                continue

            gains = self.G_function(np.array(
                list([np.bincount(
                    y_sorted[:i:],
                    minlength=self.num_class) for i in diff + 1])
            ),
                diff + 1,
                np.array(
                    list([np.bincount(
                        y_sorted[i::],
                        minlength=self.num_class) for i in diff + 1])),
                x.shape[0] - diff + 1)

            gains_all.append(np.max(gains))
            features.append(i)
            thresholds.append(x_sorted[np.argmax(gains)])

        max_gain = np.argmax(gains_all)

        if len(thresholds) != 0:
            self.feature_importances[features[max_gain]] += max_gain

        return features[max_gain], thresholds[max_gain]

    def __fit_node(self, x, y, node_id, depth):
        if (depth == self.max_depth) or \
                (x.shape[0] <= self.min_samples_split) or np.all(y == y[0]):
            self.tree[node_id] = [self.__class__.LEAF_TYPE, mode(y)[0]]
            return self.tree[node_id]

        feature_id, threshold = self.__find_threshold(x, y)
        if threshold is None:
            self.tree[node_id] = [self.__class__.LEAF_TYPE, mode(y)[0]]
            return self.tree[node_id]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        self.tree[node_id] = [self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold]

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

        return self.tree[node_id]

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.n_samples = x.shape[0]
        self.feature_importances = np.zeros(shape=x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [330]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [331]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [306]:
%time clf.fit(X_train, y_train)

CPU times: user 1.99 ms, sys: 1.33 ms, total: 3.32 ms
Wall time: 1.85 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [307]:
%time my_clf.fit(X_train, y_train)

CPU times: user 171 ms, sys: 4.7 ms, total: 176 ms
Wall time: 174 ms


## Проверка качества работы на wine

In [308]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

In [309]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

## Подготовка данных Speed Dating Data 

In [454]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',encoding = 'latin1')

In [455]:
df = df.drop(columns = df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64'].index.values).fillna(-100000)

In [456]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_ = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_ = df_.drop(['iid', 'pid'], axis=1)
X = df_.drop(['match'], axis=1).values
y = df_['match'].values

In [457]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3765, 364) (3765,) (419, 364) (419,)


## Проверка скорости работы на Speed Dating Data 

In [442]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [443]:
%time clf.fit(X_train, y_train)

CPU times: user 92.4 ms, sys: 4.5 ms, total: 96.9 ms
Wall time: 95.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [458]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.7 s, sys: 238 ms, total: 19 s
Wall time: 19.1 s


## Проверка качества работы на Speed Dating Data

In [459]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [460]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Задание 3

## Задание 4

In [461]:
print(pd.Series(index = df_.drop(['match'], axis=1).columns, data = clf.feature_importances_).
      sort_values()[::-1].head(10))

dec         0.655605
dec_o       0.344395
amb5_3_f    0.000000
sinc4_2     0.000000
shar7_2     0.000000
attr1_2     0.000000
sinc1_2     0.000000
intel1_2    0.000000
fun1_2      0.000000
amb1_2      0.000000
dtype: float64


In [462]:
print(pd.Series(index = df_.drop(['match'], axis=1).columns, data = my_clf.feature_importances).
      sort_values()[::-1].head(10))

int_corr     2324.0
attr1_1      1359.0
like          921.0
like_o        870.0
intel_o       823.0
sinc3_1       804.0
attr2_1_f     800.0
movies        748.0
intel1_1      682.0
theater_f     629.0
dtype: float64


## Задание 5

In [463]:
from sklearn.model_selection import StratifiedKFold
from scipy.stats import randint as randint

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_leaf': randint(5, 10),
    'class_weight': [None, 'balanced']}

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
model = RandomForestClassifier()

In [464]:
rs = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=123)
rs.fit(X, y)

/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a1427e908>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1a14bacf60>, 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_

In [465]:
print(rs.best_params_, rs.best_score_)

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 8} 0.9873471968816593
